<a href="https://colab.research.google.com/github/digitalhumanitiestextbook/dhtextbook/blob/main/chapter06/6_create_noun_matrix_from_txt_practice.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

- 아래 코드를 실행한 후 나타나는 "파일 선택" 버튼을 클릭하여, 분석할 txt 파일을 업로드하세요.

In [ ]:
from google.colab import files
import os

# 파일 업로드
uploaded = files.upload()

# 업로드한 파일 경로 가져오기
file_path = list(uploaded.keys())[0]
root, ext = os.path.splitext(file_path)
output_file_path = root + '_matrix.csv'

In [ ]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK 자연어 처리에 필요한 데이터 다운로드
nltk.download('punkt_tab')  # NLTK 자연어 처리에 필요한 데이터 다운로드
nltk.download('averaged_perceptron_tagger_eng')  # 품사 태깅 모델
nltk.download('wordnet')  # 표제어 추출을 위한 WordNet 사전
nltk.download('stopwords')  # 불용어 사전

stop_words = set(stopwords.words('english'))  # 영어 불용어 집합
lemmatizer = WordNetLemmatizer()  # 표제어 추출기

# 명사 추출 함수
def extract_nouns(sentence):
    sentence_string = str(sentence) # 문장을 문자열로 변환
    cleaned_sentence = re.sub(r'[^\w\s]',' ', sentence_string)  # 알파벳(숫자 포함)과 공백 외의 모든 특수문자를 빈 칸으로 대체하고 제거
    cleaned_sentence = re.sub(r'\d+', '', cleaned_sentence)  # 숫자 제거
    nouns = []  # 명사 리스트 초기화

    # 품사 태깅 및 명사 추출
    for word, pos in nltk.pos_tag(nltk.word_tokenize(cleaned_sentence)):
        if pos.startswith('NN'):  # 명사인지 확인
            word = word.lower()  # 소문자 변환
            word = lemmatizer.lemmatize(word)  # 표제어 추출: 복수형 명사 단수화

            if len(word) > 2 and word not in stop_words:  # 길이가 2보다 크고 불용어가 아닌 경우
                nouns.append(word)  # 명사 리스트에 추가

    return nouns  # 추출된 명사 리스트 반환

# 파일에서 텍스트 읽기
with open(file_path, 'r') as file:
    text = file.read()

sentences = nltk.sent_tokenize(text)  # 텍스트를 문장 단위로 분리
nouns_per_sentence = [extract_nouns(sentence) for sentence in sentences]  # 각 문장에서 명사 추출

all_nouns = set()  # 전체 명사 집합 초기화
for nouns in nouns_per_sentence:  # 각 문장에서 추출된 명사 리스트를 순회
    all_nouns.update(nouns)  # 전체 명사 집합에 추가

matrix = pd.DataFrame(0, index=list(all_nouns), columns=list(all_nouns))  # 명사간 동시출현 행렬 초기화

# 명사 동시출현 빈도계산
for nouns in nouns_per_sentence:  # 각 문장의 명사 리스트 순회
    for i, noun1 in enumerate(nouns):  # 첫 번째 명사 순회
        for noun2 in nouns[i+1:]:  # 첫 번째 명사 이후 명사들과 조합
            matrix.loc[noun1, noun2] += 1  # 동시출현 빈도 증가
            matrix.loc[noun2, noun1] += 1  # 대칭 행렬로 만들기

matrix.to_csv(output_file_path)  # CSV 파일로 저장
matrix

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,field,gaiety,mean,acre,girl,companion,lever,hatbox,pennsylvania,benediction,...,cunard,swallow,story,fur,coherence,street,taxicab,cape,venice,singing
field,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
gaiety,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mean,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acre,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
girl,0,0,0,0,8,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
street,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
taxicab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cape,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
venice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- 아래 코드를 실행하면 분석 결과가 담긴 CSV 파일이 다운로드됩니다.
- 파일명은 [업로드한 파일명]_matrix.csv 형식이며, 실행 중인 브라우저의 다운로드 폴더에서 확인할 수 있습니다.

In [ ]:
# Google Colab 환경에서 생성된 CSV 파일을 로컬로 다운로드
from google.colab import files

files.download(output_file_path)